# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.ObjectModel;
using System.IO;
using System.Linq;
using System.Text;

#r "nuget: XPlot.Plotly.Interactive"

## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


In [ ]:
public class SudokuGrid : ICloneable
{
    // Méthodes utilitaires

    // Méthode pour convertir un tableau 1D en tableau de tableaux 1D (jagged array)
    public static T[][] ToJaggedArray<T>(IList<T> source, int columnLength)
    {
        return source
            .Select((value, index) => new { value, index })
            .GroupBy(x => x.index / columnLength)
            .Select(g => g.Select(x => x.value).ToArray())
            .ToArray();
    }

    // Méthode pour aplatir un tableau 2D en un tableau 1D
    public static T[] Flatten<T>(T[][] source) => source.SelectMany(x => x).ToArray();

    // Méthode pour convertir un tableau de tableaux 1D (jagged array) en un tableau 2D
    public static T[,] To2D<T>(T[][] source)
    {
        int rowLength = source.Length;
        int colLength = source[0].Length;
        var result = new T[rowLength, colLength];

        for (int row = 0; row < rowLength; row++)
            for (int col = 0; col < colLength; col++)
                result[row, col] = source[row][col];

        return result;
    }

    // Méthode pour convertir un tableau 2D en un tableau de tableaux 1D (jagged array)
    public static T[][] ToJaggedArray<T>(T[,] source)
    {
        int rows = source.GetLength(0);
        int cols = source.GetLength(1);
        var result = new T[rows][];

        for (int i = 0; i < rows; i++)
        {
            result[i] = new T[cols];
            for (int j = 0; j < cols; j++)
                result[i][j] = source[i, j];
        }

        return result;
    }

    // Collection d'indices pour itérer sur les voisins
    public static readonly ReadOnlyCollection<int> NeighbourIndices = new(Enumerable.Range(0, 9).ToList());

    // Définition des voisins par ligne, colonne et boîte
    private static readonly (int row, int column)[][] LineNeighbours = NeighbourIndices.Select(r => NeighbourIndices.Select(c => (r, c)).ToArray()).ToArray();
    private static readonly (int row, int column)[][] ColumnNeighbours = NeighbourIndices.Select(c => NeighbourIndices.Select(r => (r, c)).ToArray()).ToArray();
    private static readonly (int row, int column)[][] BoxNeighbours = GetBoxNeighbours();
    public static readonly (int row, int column)[][] AllNeighbours = LineNeighbours.Concat(ColumnNeighbours).Concat(BoxNeighbours).ToArray();

    // Calcul des voisins par boîte
    private static (int row, int column)[][] GetBoxNeighbours()
    {
        var result = new (int row, int column)[9][];

        for (int box = 0; box < 9; box++)
        {
            var cells = new List<(int, int)>();
            int startRow = (box / 3) * 3;
            int startCol = (box % 3) * 3;

            for (int row = startRow; row < startRow + 3; row++)
                for (int col = startCol; col < startCol + 3; col++)
                    cells.Add((row, col));

            result[box] = cells.ToArray();
        }

        return result;
    }

    // Tableau des voisins pour chaque cellule
    public static readonly (int row, int column)[][][] CellNeighbours = NeighbourIndices
        .Select(row => NeighbourIndices
            .Select(col => AllNeighbours
                .Where(neighbourhood => neighbourhood.Contains((row, col)))
                .SelectMany(n => n)
                .Where(pos => pos != (row, col))
                .Distinct()
                .ToArray())
            .ToArray())
        .ToArray();

    // Constructeur par défaut
    public SudokuGrid() { }

    // Grille de Sudoku
    public int[,] Cells { get; set; } = new int[9, 9];

    // Représentation de la grille sous forme de chaîne de caractères
    public override string ToString()
    {
        var output = new StringBuilder();
        var lineSep = new string('-', 31);
        var blankSep = new string(' ', 8);

        for (int row = 0; row < 9; row++)
        {
            output.Append(row % 3 == 0 ? lineSep + "\n" : "");
            output.Append("| ");
            for (int col = 0; col < 9; col++)
            {
                output.Append(Cells[row, col] > 0 ? Cells[row, col].ToString() : " ");
                output.Append((col + 1) % 3 == 0 ? " | " : "  ");
            }
            output.Append("\n");
        }
        output.Append(lineSep);

        return output.ToString();
    }

    // Méthode pour obtenir les numéros disponibles pour une cellule donnée
    public int[] GetAvailableNumbers(int x, int y)
    {
        if (x < 0 || x >= 9 || y < 0 || y >= 9)
            throw new ApplicationException("Invalid Coordinates");

        bool[] used = new bool[9];
        foreach (var (row, col) in CellNeighbours[x][y])
        {
            int value = Cells[row, col];
            if (value > 0) used[value - 1] = true;
        }

        return Enumerable.Range(1, 9).Where(n => !used[n - 1]).ToArray();
    }

    // Lecture d'un Sudoku à partir d'une chaîne de caractères
    public static SudokuGrid ReadSudoku(string sudokuAsString) => ReadMultiSudoku(new[] { sudokuAsString })[0];

    // Lecture de plusieurs grilles de Sudoku à partir d'un fichier
    public static List<SudokuGrid> ReadSudokuFile(string fileName) => ReadMultiSudoku(File.ReadAllLines(fileName));

    // Lecture de plusieurs grilles de Sudoku à partir d'un tableau de chaînes de caractères
    public static List<SudokuGrid> ReadMultiSudoku(string[] lines)
    {
        var grids = new List<SudokuGrid>();
        var rows = new List<int[]>();
        var rowCells = new List<int>();

        foreach (var line in lines.Where(l => !string.IsNullOrWhiteSpace(l)))
        {
            foreach (char c in line)
            {
                if (IsSudokuChar(c))
                {
                    rowCells.Add(char.IsDigit(c) ? (int)char.GetNumericValue(c) : 0);
                }

                if (rowCells.Count == 9)
                {
                    rows.Add(rowCells.ToArray());
                    rowCells.Clear();
                }

                if (rows.Count == 9)
                {
                    grids.Add(new SudokuGrid { Cells = To2D(rows.ToArray()) });
                    rows.Clear();
                }
            }
        }

        return grids;
    }

    // Vérification si un caractère est valide pour un Sudoku
    private static bool IsSudokuChar(char c) => char.IsDigit(c) || c == '.' || c == 'X' || c == '-';

    // Clone de la grille de Sudoku
    public object Clone() => new SudokuGrid { Cells = (int[,])Cells.Clone() };

    // Calcul du nombre d'erreurs par rapport à une grille originale
    public int NbErrors(SudokuGrid originalPuzzle)
    {
        int errors = AllNeighbours.Select(n => n.Select(pos => Cells[pos.row, pos.column])
                                                 .GroupBy(val => val)
                                                 .Sum(g => g.Count() - 1))
                                   .Sum();

        foreach (var row in NeighbourIndices)
            foreach (var col in NeighbourIndices)
                if (originalPuzzle.Cells[row, col] > 0 && originalPuzzle.Cells[row, col] != Cells[row, col])
                    errors++;

        return errors;
    }

    // Vérification de la validité de la grille par rapport à une grille originale
    public bool IsValid(SudokuGrid originalPuzzle) => NbErrors(originalPuzzle) == 0;

    // Calcul du nombre de cellules vides
    public int NbEmptyCells() => Cells.Cast<int>().Count(c => c == 0);
}


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


In [ ]:
public interface ISudokuSolver
{
    SudokuGrid Solve(SudokuGrid s);
}


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger  des grilles de Sudoku et tester des solvers.

- `GetSudokus` : Renvoie des listes de Sudoku issues de fichiers de 3 difficultés différentes.
- `SolveSudoku` : effectue un test simple d'un solver sur un sudoku donné.
- `TestSolvers` : exécute les tests de performance sur plusieurs solveurs.
- `DisplayResults` : affiche les résultats des tests sous forme de graphiques.



In [ ]:
using System.IO;
using System.Diagnostics;
using System.Threading;
using System.Threading.Tasks;
using XPlot.Plotly;
using Trace = XPlot.Plotly.Trace;
using Microsoft.DotNet.Interactive;

public enum SudokuDifficulty
{
    Easy,
    Medium,
    Hard
}

public static class SudokuHelper
{
    private const string PUZZLES_FOLDER_NAME = "Puzzles";

    // Charge les sudokus à partir des fichiers
    public static List<SudokuGrid> GetSudokus(SudokuDifficulty difficulty)
    {
        string fileName = difficulty switch
        {
            SudokuDifficulty.Easy => "Sudoku_Easy51.txt",
            SudokuDifficulty.Medium => "Sudoku_hardest.txt",
            _ => "Sudoku_top95.txt"
        };

        var currentDirectory = new DirectoryInfo(Environment.CurrentDirectory);
        DirectoryInfo puzzlesDirectory = null;

        while (puzzlesDirectory == null)
        {
            puzzlesDirectory = currentDirectory.GetDirectories()
                                               .FirstOrDefault(d => d.Name == PUZZLES_FOLDER_NAME);
            currentDirectory = currentDirectory.Parent;
            if (currentDirectory == null)
                throw new ApplicationException("Couldn't find puzzles directory");
        }

        string filePath = Path.Combine(puzzlesDirectory.ToString(), fileName);
        return SudokuGrid.ReadSudokuFile(filePath);
    }

    // Résout un sudoku avec un solver donné
    public static SudokuGrid SolveSudoku(SudokuGrid sudoku, ISudokuSolver solver)
    {
        display($"Résolution par le solver {solver.GetType().Name} du Sudoku:\n {sudoku}");
        var stopwatch = System.Diagnostics.Stopwatch.StartNew();
        var solvedSudoku = solver.Solve(sudoku);
        stopwatch.Stop();
        display($"Sudoku renvoyé:\n{solvedSudoku}\nNombre d'erreurs réstantes: {solvedSudoku.NbErrors(sudoku)}\nTemps de résolution: {stopwatch.Elapsed.TotalMilliseconds} ms");
        return solvedSudoku;
    }

    // Teste les solveurs sur des sudokus de difficulté croissante
    public static List<(string SolverName, string Difficulty, double Time, int SolvedCount, string Status)> TestSolvers(List<(string Name, ISudokuSolver Solver)> solvers, 
    int numberOfSudokus = 10, int timeLimitMilliseconds = 3000)
    {
        var results = new List<(string SolverName, string Difficulty, double Time, int SolvedCount, string Status)>();
        var difficulties = new[] { SudokuDifficulty.Easy, SudokuDifficulty.Medium, SudokuDifficulty.Hard };

        var displayPlaceholder = display("Running tests...");

        foreach (var (solverName, solver) in solvers)
        {
            foreach (var difficulty in difficulties)
            {
                var sudokus = GetSudokus(difficulty).Take(numberOfSudokus).ToList();
                Stopwatch stopwatch = new Stopwatch();
                int solvedCount = 0;
                string status = "Success";

                var message = $"Testing {solverName} on {difficulty} sudokus...";
                displayPlaceholder.Update(message);
                foreach (var sudoku in sudokus)
                {
                    var cts = new CancellationTokenSource();
                    cts.CancelAfter(timeLimitMilliseconds);

                    Task task = Task.Run(() =>
                    {
                        try
                        {
                            SudokuGrid solved = solver.Solve(sudoku);
                            if (solved.NbErrors(sudoku) == 0)
                            {
                                Interlocked.Increment(ref solvedCount);
                            }
                        }
                        catch (Exception)
                        {
                            // Ignore exceptions for unsolvable sudokus
                        }
                    }, cts.Token);

                    stopwatch.Start();
                    if (!task.Wait(timeLimitMilliseconds))
                    {
                        status = "Timeout";
                        break;
                    }
                    stopwatch.Stop();

                    if (cts.Token.IsCancellationRequested)
                    {
                        status = "Timeout";
                        break;
                    }
                }

                if (solvedCount < numberOfSudokus)
                {
                    status = "Disqualified";
                }

                double totalTime = stopwatch.Elapsed.TotalMilliseconds;
                results.Add((solverName, difficulty.ToString(), totalTime, solvedCount, status));
            }
        }

        return results;
    }

    // Affiche les résultats des tests en historgrammes avec Plotly
    public static void DisplayResults(List<(string SolverName, string Difficulty, double Time, int SolvedCount, string Status)> results)
    {
        var solverNames = results.Where(r => r.Status != "Disqualified").Select(r => r.SolverName).Distinct().ToArray();
        var difficulties = new[] { "Easy", "Medium", "Hard" };

        foreach (var difficulty in difficulties)
        {
            var difficultyResults = results.Where(r => r.Difficulty == difficulty && r.Status != "Disqualified").ToList();
            var trace = new List<Trace>
            {
                new Bar
                {
                    x = difficultyResults.Select(r => r.SolverName).ToArray(),
                    y = difficultyResults.Select(r => r.Time).ToArray(),
                    name = difficulty
                }
            };

            var layout = new Layout.Layout
            {
                title = $"Comparison of Solver Performance - {difficulty} Difficulty",
                xaxis = new Xaxis { title = "Solver" },
                yaxis = new Yaxis { title = "Total Time (ms)", type = "log" }
            };

            var chart = Chart.Plot(trace);
            chart.WithLayout(layout);
            display(chart);
        }
    }

}
